<a href="https://colab.research.google.com/github/ygordev/Flask/blob/main/Flask_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Flask - Code Snippets**
<br>por Ygor Moreira Lima @colt7r

---

Exemplo de Estrutura de Arquivos<br>
<u>PROJETO</u><br>
static/<br>
----------- arquivo.css<br>
----------- arquivo.js<br>
templates/<br>
----------- index.html<br>
app.py<br>




**Referências**

https://flask.palletsprojects.com/en/2.0.x/errorhandling/

https://realpython.com/flask-blueprint/

https://flask.palletsprojects.com/en/1.1.x/blueprints/

https://www.geeksforgeeks.org/how-to-run-flask-app-on-google-colab/

**Backlog**

*   Consertar "Excluir id=5 do BANCO DE DADOS usando Blueprint"




In [ ]:
# Criação dos arquivos template

# Exemplo de uso de loops nos templates:
# { % for i in range(0,10) % }
# { { i }}
# { % endfor % }

# Exemplo de uso de condicionais nos templates:
# { % if 3 > 4 % }
# { É maior que 4! }
# { % else % }
# { % Não é maior que 4! % }
# { % endif % }


# formulario_get.html
with open('templates/formulario_get.html', 'w') as formulario_get:
  corpo = """
  <html><body><h1>Olá, {{ meu_nome }}!</h1><br><h2>Método GET</h2>
  <form action="{{ url_for('receber') }}" method="GET">
  Nome: <input type="text" name="nome"/><br>
  Idade: <input type="text" name="idade"/><br>
	<br><br>
	<input type="submit" value="Enviar"/>
  </form>
  </body>
  </html>
  """
  formulario_get.write(corpo)

# formulario_post.html
with open('templates/formulario_post.html', 'w') as formulario_post:
  corpo = """
  <html><body><h1>Método POST</h1>
  <form action="{{ url_for('receber') }}" method="POST">
  Nome: <input type="text" name="nome"/><br>
  Idade: <input type="text" name="idade"/><br>
	<br><br>
	<input type="submit" value="Enviar"/>
  </form>
  </body>
  </html>
  """
  formulario_post.write(corpo)

# Criação de um arquivo Blueprint para excluir o item no BD com id = 1
with open('requisicao.py', 'w') as blueprint_operacoesCRUD:
  corpo = """
from flask import Flask, render_template, request, session, redirect, url_for, Blueprint
import dataset

bp_requisicao = Blueprint('requisicao', __name__)
# O decorator deve ir com o nome do blueprint.
# Ex: @bp_requisicao.route, ao invés de @app.route
  
@bp_requisicao.route("/exclusao", methods=['GET'])
def exclusao_banco():
  return render_template('blueprint_resposta.html'), 200

# with dataset.connect("sqlite:///cursopython.db") as db:
# Como excluir dados [DELETE]
# db['aulas'].delete(id=5)"""
  blueprint_operacoesCRUD.write(corpo)

In [ ]:
from flask import Flask, render_template, request, session, redirect, url_for
from flask import jsonify, Blueprint
from flask_ngrok import run_with_ngrok
import dataset

# Importando a blueprint 'requisicao'
from requisicao import bp_requisicao

app = Flask(__name__)
app.secret_key = 'MINHA_CHAVE_CRIPTOGRAFADA' # Para a utilização nas SESSIONS
app.register_blueprint(bp_requisicao)
run_with_ngrok(app)   
  
######## Definição de uma rota simples ######
@app.route("/")
def home():
  html = """
  <html><body><h1>Estou em Home!</h1><br>
  <a href="{}">Acessar Formulário GET</a><br>
  <a href="{}">Acessar Formulário POST</a><br>
  <a href="{}">Criar SESSION</a><br>
  <a href="{}">Excluir SESSION</a><br>
  <a href="{}">Operar BANCO DE DADOS</a><br>
  <a href="{}">Listar BANCO DE DADOS</a><br>
  <a href="{}">Excluir id=5 do BANCO DE DADOS usando Blueprint</a><br>
  """.format(url_for('formulario_get_method'), url_for('formulario_post_method'), url_for('acesso_session'), url_for('sair_session'), url_for('operacao_banco'), url_for('listar_banco'), url_for('requisicao.exclusao_banco'))
  return html, 200

######## Tratamento de informações passadas por formulário #######
# GET rota/?nome=Ygor&idade=28
# POST não aparece nada na URL
@app.route("/formulario_get")
def formulario_get_method():
  # Todos os parâmetros recebidos pelo template HTML devem estar entre colchetes. 
  # Dentro do formulario_get.html, por exemplo: <h1>Olá, {{ meu_nome }}!</h1>
  return render_template('formulario_get.html', meu_nome="Ygor Lima"), 200

@app.route("/formulario_post", methods=['GET', 'POST'])
def formulario_post_method():
  return render_template('formulario_post.html'), 200

@app.route("/receber/", methods=['GET', 'POST'])
def receber():
	if request.method == "GET":
		return "Estou em GET<br><br>Nome: {}<br>Idade: {}<br>".format(request.args.get("nome"), request.args.get("idade")), 200
	elif request.method == "POST":
		return "Estou em POST<br><br>Nome: {}<br>Idade: {}<br>".format(request.form["nome"], request.form["idade"]), 200

######## Tratamento de informações passadas por URL #######
# Tipos possíveis de declaração na rota:
# String rota/Ygor
# int    rota/123
# Float. rota/123.45
# Path.  rota/url/do/site/arquivo.html
############################################################
@app.route("/informacao")
@app.route("/informacao/<nome>")
@app.route("/informacao/<nome>/<idade>")
def info(nome = None, idade = None):
	return "Nome: {}<br>Idade: {}<br>".format(nome, idade), 200

####### Utilização de SESSIONS ######
@app.route("/session", methods=['GET', 'POST'])
def acesso_session():
	return """
		<html><body><h1>Início da Sessão</h1>
		<form action="{}" method="POST">
		Usuário: <input type="text" name="usuario"/><br>
		<input type="submit" value="Acesso Restrito!"/> 
		</form>
		""".format(url_for('validacao_sessao')), 200

@app.route("/validacao/", methods=['POST'])
def validacao_sessao():
	if request.method == 'POST':
		session['usuario'] = request.form["usuario"]
		return redirect(url_for('acesso_restrito'))
	else:
		return redirect(url_for('acesso_session')), 200

@app.route("/restrito/")
def acesso_restrito():
	if (session['usuario']):
		return "Estou na área de acesso restrito!<br>Usuário logado: {}".format(session['usuario']), 200
	else:
		return redirect(url_for('acesso_session')), 200

@app.route("/sair/")
def sair_session():
	print(session)
	session.pop('usuario', None)
	return redirect(url_for('acesso_session')), 200

###### Operações CRUD ######
@app.route("/operacao/")
def operacao_banco():
	with dataset.connect("sqlite:///cursopython.db") as db:
		# Como inserir dados [CREATE]
		db['aulas'].insert(dict(nome="Aula 1", tipo="Python"))
		db['aulas'].insert(dict(nome="Aula 2", tipo="Banco de dados"))
		db['aulas'].insert(dict(nome="Aula 3", tipo="HTML"))
		db['aulas'].insert(dict(nome="Aula 4", tipo="Javascript"))

		# Como ler dados [READ]
		#lista = db['aulas'].all()
		#lista = db['aulas'].find(_limit=2, order_by='nome', _offset=2)
		#lista = db['aulas'].find_one()
		#lista = db['aulas'].find_one(nome="Aula 1")

		# Como atualizar dados [UPDATE]
		#lista['tipo'] = "Python ALTERADO"
		#db['aulas'].update(lista, ['id'])

		# Como excluir dados [DELETE]
		#db['aulas'].delete(id=1)

	return "Realizando operações CRUD!", 200

@app.route("/banco/")
def listar_banco():
	with dataset.connect("sqlite:///cursopython.db") as db:
		lista = db['aulas'].all()

	html = "<ul>"
	for item in lista:
		html += "<li>'id': {id} - 'nome': {nome} - 'tipo': {tipo}</li>".format(id=item['id'],nome=item['nome'],tipo=item['tipo'])
	html += "</ul>"

	return "Estou no banco de dados!<br>Lista: <br>{}".format(html), 200 

###### Utilização de BLUEPRINTS ######
# Blueprints são projetos Flask separados em arquivos e que podem ter funcionalidades complementares e únicas.
# Cada Blueprint pode ser usado em camadas e isso te ajuda a organizar melhor o seu projeto.
# Para utilizar:
# 1 - Importar Blueprint em ambos os arquivos: from flask import Blueprint
# 2 - Inicializar: bp_requisicao = Blueprint('NOME', __name__, url_prefix='/url/para/o/blueprint') url_prefix é opcional, pois posso definir por rotas também
# 3 - Importar no projeto principal: from requisicao import bp_requisicao (importar do arquivo blueprint o construtor)
# 4 - Registrar a blueprint: app.register_blueprint(bp_requisicao)

###### Criação de API RESTful #######
# Ver arquivo Flask-API.ipynb
 
###### Inicialização do servidor Flask ####### 
print(app.url_map)
app.run()

Map([<Rule '/formulario_post' (POST, GET, HEAD, OPTIONS) -> formulario_post_method>,
 <Rule '/formulario_get' (GET, HEAD, OPTIONS) -> formulario_get_method>,
 <Rule '/informacao' (GET, HEAD, OPTIONS) -> info>,
 <Rule '/validacao/' (POST, OPTIONS) -> validacao_sessao>,
 <Rule '/exclusao/' (GET, HEAD, OPTIONS) -> requisicao.exclusao_banco>,
 <Rule '/restrito/' (GET, HEAD, OPTIONS) -> acesso_restrito>,
 <Rule '/operacao/' (GET, HEAD, OPTIONS) -> operacao_banco>,
 <Rule '/receber/' (POST, GET, HEAD, OPTIONS) -> receber>,
 <Rule '/session' (POST, GET, HEAD, OPTIONS) -> acesso_session>,
 <Rule '/banco/' (GET, HEAD, OPTIONS) -> listar_banco>,
 <Rule '/sair/' (GET, HEAD, OPTIONS) -> sair_session>,
 <Rule '/' (GET, HEAD, OPTIONS) -> home>,
 <Rule '/informacao/<nome>/<idade>' (GET, HEAD, OPTIONS) -> info>,
 <Rule '/informacao/<nome>' (GET, HEAD, OPTIONS) -> info>,
 <Rule '/static/<filename>' (GET, HEAD, OPTIONS) -> static>])
 * Serving Flask app "__main__" (lazy loading)
 * Environment: producti

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6d932fde3c43.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
